In [2]:
import pandas as pd
import numpy as np

In [4]:
#loading train and test data

train_data=pd.read_csv('B:/spyder/risk_analytics_train.csv',header=0)
test_data=pd.read_csv('B:/spyder/risk_analytics_test.csv',header=0)

In [5]:
train_data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0.0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1.0,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0.0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0.0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0.0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [7]:
train_data.shape

(614, 13)

In [14]:
test_data.shape

(367, 12)

In [6]:
print(train_data.isnull().sum())

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64


In [8]:
#handling the missing value of categorical data by replacing with mode.
colname1=["Gender","Married","Dependents","Self_Employed","Loan_Amount_Term"]
for x in colname1[:]:
    train_data[x].fillna(train_data[x].mode()[0],inplace=True)

In [9]:
print(train_data.isnull().sum())


Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term      0
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64


In [10]:
#inputing the numerical missing value using mean
train_data["LoanAmount"].fillna(train_data["LoanAmount"].mean(),inplace=True)

In [11]:
print(train_data.isnull().sum())


Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64


In [12]:
#imputing the value for credit history coilumn differently
train_data['Credit_History'].fillna(value=0,inplace=True)
print(train_data.isnull().sum())

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64


In [13]:
#transforming categorical data to numerical
from sklearn import preprocessing

colname=["Gender","Married","Education","Self_Employed","Property_Area","Loan_Status"]
le={}

for x in colname:
    le[x]=preprocessing.LabelEncoder()
for x in colname:
    train_data[x]=le[x].fit_transform(train_data.__getattr__(x))
#convert loan status as y=1 and n=0

In [15]:
#creating training and testing and running the model.

X_train=train_data.values[:,1:-1]
Y_train=train_data.values[:,-1]
Y_train=Y_train.astype(int)

In [17]:
from sklearn.model_selection import train_test_split

x_train , x_test  , y_train , y_test = train_test_split(X_train , Y_train , 
                                                        test_size = 0.3 ,
                                                        random_state = 10)

In [23]:
x_test.shape
x_train.shape

(185, 11)

In [18]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

scaler.fit(x_train)
x_train=scaler.transform(x_train)


scaler.fit(x_test)
x_test=scaler.transform(x_test)

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [19]:
from sklearn import svm
svc_model=svm.SVC(kernel='rbf',C=1.0,gamma=0.1)
svc_model.fit(x_train,y_train)


In [24]:
##Predict on train data
Y_pred=svc_model.predict(x_train)

In [25]:
from sklearn.metrics import confusion_matrix , accuracy_score , classification_report
print(confusion_matrix(y_train,Y_pred))
print(accuracy_score(y_train,Y_pred))
print(classification_report(y_train,Y_pred))

[[ 83  57]
 [ 24 265]]
0.811188811189
             precision    recall  f1-score   support

          0       0.78      0.59      0.67       140
          1       0.82      0.92      0.87       289

avg / total       0.81      0.81      0.80       429



In [26]:
###Predict on test data for validation
Y_pred=svc_model.predict(x_test)

In [27]:
from sklearn.metrics import confusion_matrix , accuracy_score , classification_report
print(confusion_matrix(y_test,Y_pred))
print(accuracy_score(y_test,Y_pred))
print(classification_report(y_test,Y_pred))

[[ 18  34]
 [ 16 117]]
0.72972972973
             precision    recall  f1-score   support

          0       0.53      0.35      0.42        52
          1       0.77      0.88      0.82       133

avg / total       0.71      0.73      0.71       185



In [36]:
####Now use XGBoost 

from xgboost import XGBClassifier
classifier=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
classifier.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [37]:
Y_pred=classifier.predict(x_train)

In [38]:
from sklearn.metrics import confusion_matrix , accuracy_score , classification_report
print(confusion_matrix(y_train,Y_pred))
print(accuracy_score(y_train,Y_pred))
print(classification_report(y_train,Y_pred))

[[ 76  64]
 [ 25 264]]
0.792540792541
             precision    recall  f1-score   support

          0       0.75      0.54      0.63       140
          1       0.80      0.91      0.86       289

avg / total       0.79      0.79      0.78       429



In [39]:
Y_pred=classifier.predict(x_test)

In [40]:
from sklearn.metrics import confusion_matrix , accuracy_score , classification_report
print(confusion_matrix(y_test,Y_pred))
print(accuracy_score(y_test,Y_pred))
print(classification_report(y_test,Y_pred))

[[ 19  33]
 [ 14 119]]
0.745945945946
             precision    recall  f1-score   support

          0       0.58      0.37      0.45        52
          1       0.78      0.89      0.84       133

avg / total       0.72      0.75      0.73       185



In [41]:
####Now use logistic regresiion
from sklearn.linear_model import LogisticRegression

#creating model 
classifier_lr = (LogisticRegression())

#fitting  training data to model
classifier_lr.fit(x_train , y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [42]:
Y_pred=classifier_lr.predict(x_train)

In [43]:
from sklearn.metrics import confusion_matrix , accuracy_score , classification_report
print(confusion_matrix(y_train,Y_pred))
print(accuracy_score(y_train,Y_pred))
print(classification_report(y_train,Y_pred))

[[ 76  64]
 [ 32 257]]
0.776223776224
             precision    recall  f1-score   support

          0       0.70      0.54      0.61       140
          1       0.80      0.89      0.84       289

avg / total       0.77      0.78      0.77       429



In [48]:
Y_pred=classifier_lr.predict(x_test)

In [49]:
from sklearn.metrics import confusion_matrix , accuracy_score , classification_report
print(confusion_matrix(y_test,Y_pred))
print(accuracy_score(y_test,Y_pred))
print(classification_report(y_test,Y_pred))

[[ 20  32]
 [ 12 121]]
0.762162162162
             precision    recall  f1-score   support

          0       0.62      0.38      0.48        52
          1       0.79      0.91      0.85       133

avg / total       0.74      0.76      0.74       185



In [46]:
from sklearn.tree import DecisionTreeClassifier

model_DecisionTree=DecisionTreeClassifier()
model_DecisionTree.fit(x_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [50]:
Y_pred=model_DecisionTree.predict(x_train)

In [51]:
from sklearn.metrics import confusion_matrix , accuracy_score , classification_report
print(confusion_matrix(y_train,Y_pred))
print(accuracy_score(y_train,Y_pred))
print(classification_report(y_train,Y_pred))

[[140   0]
 [  0 289]]
1.0
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       140
          1       1.00      1.00      1.00       289

avg / total       1.00      1.00      1.00       429



In [52]:
Y_pred=model_DecisionTree.predict(x_test)

In [53]:
from sklearn.metrics import confusion_matrix , accuracy_score , classification_report
print(confusion_matrix(y_test,Y_pred))
print(accuracy_score(y_test,Y_pred))
print(classification_report(y_test,Y_pred))

[[26 26]
 [42 91]]
0.632432432432
             precision    recall  f1-score   support

          0       0.38      0.50      0.43        52
          1       0.78      0.68      0.73       133

avg / total       0.67      0.63      0.65       185



In [54]:
###As the efficiancy of SVM model is quite good so we use this model for prediction for test data

In [55]:
#preprocessing the test data
test_data.head()
print(test_data.isnull().sum())

Loan_ID               0
Gender               11
Married               0
Dependents           10
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
dtype: int64


In [56]:
#handling the missing value of test data
colname2=["Gender","Dependents","Self_Employed","Loan_Amount_Term"]
for x in colname2[:]:
    test_data[x].fillna(test_data[x].mode()[0],inplace=True)
print(test_data.isnull().sum())

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      0
Credit_History       29
Property_Area         0
dtype: int64


In [57]:
test_data["LoanAmount"].fillna(test_data["LoanAmount"].mean(),inplace=True)
print(test_data.isnull().sum())

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History       29
Property_Area         0
dtype: int64


In [60]:
#imputing the value for credit history coilumn differently
test_data['Credit_History'].fillna(value=0,inplace=True)
print(test_data.isnull().sum())

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64


In [61]:
from sklearn import preprocessing

colname3=["Gender","Married","Education","Self_Employed","Property_Area"]
le={}

for x in colname3:
    le[x]=preprocessing.LabelEncoder()
for x in colname3:
    test_data[x]=le[x].fit_transform(test_data.__getattr__(x))

In [62]:
#test_data.head()
X_test=test_data.values[:,1:]

In [63]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

scaler.fit(X_test)
X_test=scaler.transform(X_test)

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [68]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

scaler.fit(X_train)
X_train=scaler.transform(X_train)

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [64]:
Y_pred=svc_model.predict(X_test)
print(list(Y_pred))
Y_pred_col=list(Y_pred)

[1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 

In [66]:
test_data=pd.read_csv('B:/spyder/risk_analytics_test.csv',header=0)
test_data["Y_prediction"]=Y_pred_col
test_data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Y_prediction
0,LP001015,Male,Yes,0.0,Graduate,No,5720,0,110.0,360.0,1.0,Urban,1
1,LP001022,Male,Yes,1.0,Graduate,No,3076,1500,126.0,360.0,1.0,Urban,1
2,LP001031,Male,Yes,2.0,Graduate,No,5000,1800,208.0,360.0,1.0,Urban,1
3,LP001035,Male,Yes,2.0,Graduate,No,2340,2546,100.0,360.0,NaN,Urban,0
4,LP001051,Male,No,0.0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban,1


In [67]:
test_data.to_csv('test_data.csv')


In [69]:
svc_model=svm.SVC(kernel='rbf',C=1,gamma=0.1)

from sklearn import cross_validation
#performing the kfold cross_validation

kfold_cv=cross_validation.KFold(n=len(X_train),n_folds=10)
print(kfold_cv)

#running the model using scoring as accuracy
kfold_cv_result=cross_validation.cross_val_score(estimator=svc_model,X=X_train,y=Y_train,
                                                 scoring="accuracy",cv=kfold_cv)
print(kfold_cv_result)
#finding the mean
print(kfold_cv_result.mean())

sklearn.cross_validation.KFold(n=614, n_folds=10, shuffle=False, random_state=None)
[ 0.74193548  0.82258065  0.75806452  0.72580645  0.7704918   0.67213115
  0.75409836  0.75409836  0.78688525  0.80327869]
0.758937070333


C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
